# 텍스트와 바이트

## 4.1 문자 문제

파이썬 3 str에서는 항상 유니코드 문자 항목으로 가져온다

- 유니코드 표준에서 'U+' 접두사를 붙혀 4자리에서 6자리 사이의 16진수로 표현한다
- 문자를 표현하는 실제 바이트는 사용하는 인코딩에 따라 달라진다 인코딩은 코드 포인트를 바이트 시퀀스로 변환하는 알고리즘이다

In [1]:
# 예제 4-1 인코딩과 디코딩
s = 'café'
print(len(s)) # s는 4개의 유니코드 문자를 갖고있다

b = s.encode('utf-8') # UTF-8을 이용해서 str을 bytes로 인코딩한다
print(b) # byters 리터럴은 접두사 b로 시작한다
print(len(b)) # bytes 형인 b는 다섯 바이트로 구성된다. é가 UTF-8에서 두 바이트로 인코딩된다

print(b.decode('utf-8')) # UTF-8 인코딩을 이용해서 bytes를 str로 디코딩한다

4
b'caf\xc3\xa9'
5
café


## 4.2 바이트에 대한 기본 지식

In [2]:
# 예제 4-2 bytes와 byearray로 지정한 5바이트 시퀀스
cafe = bytes('café', encoding='utf-8') # bytes는 str에 인코딩을 지정해서 만들 수 있다
print(cafe)
print(cafe[0]) # c가 ASCII코드 기준 99이다
print(cafe[:1]) # bytes는 슬라이싱 해도 bytes이다
cafe_arr = bytearray(cafe)
print(cafe_arr) # bytearray에 대한 리터럴 구문은 없다. bytes를 리터럴 인수로 사용해서 bytearray()를 표현한다
print(cafe_arr[-1:]) # bytearray는 슬라이싱해도 bytearray이다

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


__byte_type[0] != byte_type[:1] 인 사실을 명심하자 str의 경우에만 s[0] == s[:1] 이 성립한다__

이진 시퀀스는 바이트 값에 따라 다음 형태로 출력한다
- 화면에 출력 가능한 아스키 문자는 (' ' 부터 '~')는 아스키 문자 그대로 출력한다
- 탭, 개행 문자, 캐리지 리턴, 백슬래시는 (\t, \n, \r, \\)로 출력한다
- 그 외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력한다

그렇기 때문에 café가 b'caf\xc3\xa9'가 된 것이다. (caf는 아스키코드로 표현 가능, é는 불가하기 때문에 이스케이프 시퀀스로 포현)

bytes와 bytes는 포매팅하는 format()과 format_map()을 메서드를 제외하고는 str이 제공하는 모든 메서드를 지원하고, 추가로 유니코드 데이터 관련 메서드 역시 지원한다

In [3]:
# fromhex()라는 str에는 없는 클래스 메서드 (공백으로 구분된 16진수 쌍을 파싱해서 이진 시퀀스를 만들 수 있다)
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [4]:
# 예제 4-3 배열의 원시 데이터에서 bytes 초기화하기
import array

numbers = array.array('h', [-2, -1, 0, 1, 2]) # 'h'타입코드는 short int(16비트) 형의 배열을 생성한다
octets = bytes(numbers) # octets는 number를 구성하는 바이트들의 사본을 가지고있다
octets # 다섯개의 shor int 형을 나타내는 10 바이트이다

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

### 4.2.1 구조체와 메모리 뷰

In [5]:
# 예제 4-4 memoryview와 struct를 사용해서 GIF 이미지 헤더 조사하기
import struct
fmt = '<3s3sHH' # struct 포맷을 지정한다 <: 리틀엔디언, 3s3s는 3바이트 시퀀스 두 개, HH는 16비트 정수 두 개를 나타낸다
with open('test.gif', 'rb') as fp:
    img = memoryview(fp.read()) # 메모리에 로딩된 파일 내용으로부터 memoryview를 생성한다

header = img[:10] # 먼저 생성한 memoryview를 슬라이싱해서 새로운 memeoryview를 만든다. 이 때 아무런 바이트도 복사하지 않는다
print(bytes(header)) # 화면에 출력하기 위해 bytes로 변환한다. 이 때 10 바이트가 복사된다
print(struct.unpack(fmt, header)) # memoryview를 튜플로 언패킹한다 (종류, 버전, 너비 높이)
del header # 객체에 연결된 메모리를 해제하기 위해 참조를 삭제한다
del img

b'GIF89a5\x05\xee\x02'
(b'GIF', b'89a', 1333, 750)


## 4.3 기본 인코더/디코더

In [6]:
# 예제 4-5 전혀 다른 바이트 시퀀스를 만든느 세 개의 코덱으로 인코딩한 El Niño 문자열
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4.4 인코딩/디코딩 문제 이해하기

### 4.4.1 UnicodeEncodeError 처리하기

In [7]:
# 예제 4-6 바이트로 인코딩하기: 성공 및 에러 처리
city = 'São Paulo'

"""
utf_? 계열 인코딩은 모든 str을 처리할 수 있다
iso8859_1도 São Paulo 문자열을 처리할 수 있다
"""
for codec in ['utf_8', 'utf_16', 'iso8859_1', 'cp437']:
    try:
        print(f'{codec}\t{city.encode(codec)}')
    except UnicodeEncodeError:
        print("UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>") # cp437은 ã를 인코딩 할 수 없다 UnicodeEncodeError 발생시킴

print(city.encode('cp437', errors='ignore')) # errors='ignore' 처리기는 인코딩 할 수 없는 문자를 건너뛴다
print(city.encode('cp437', errors='replace')) # 'replace' 처리기는 인코딩 할 수 없는 문자를 ?로 치환한다
print(city.encode('cp437', errors='xmlcharrefreplace')) # xmlcharrefreplace 처리기는 인코딩할 수 없는 문자를 XML 개체로 치환한다

utf_8	b'S\xc3\xa3o Paulo'
utf_16	b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
iso8859_1	b'S\xe3o Paulo'
UnicodeEncodeError: 'charmap' codec can't encode character 'ã' in position 1: character maps to <undefined>
b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


### 4.4.2 UnicodeDecodeError 처리하기

In [8]:
# 예제 4-7 str에서 bytes로 디코딩하기: 성공 및 에러 처리
octets = b'Montr\xe9al' # latin1로 인코딩된 Montréal이다. \xe9는 é를 나타내는 바이트
print(octets.decode('cp1252')) # 'cp1252'는 latin1의 슈퍼셋이므로 제대로 디코딩 된다
print(octets.decode('iso8859_7')) # ISO-8859-7은 그리스어를 위한 코덱이므로 엉뚱하게 해석한다 (에러는 x)
print(octets.decode('koi8_r')) # KOI8-R은 러시아어를 위한 코덱이여서 И으로 해석

try:
    print(octets.decode('utf_8')) # 'utf_8' 코덱으로는 변환할 수 없기 때문에 UnicodeDecodeError 발생
except UnicodeDecodeError:
    print("UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte")

print(octets.decode('utf-8', errors='replace')) # 'replace' 처리기를 사용하여 �로 치환한다

Montréal
Montrιal
MontrИal
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte
Montr�al


### 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError

In [9]:
# coding: cp1252
# 예제 4-8 ola.py: 'Hello, World!'포르투갈 버전

print('Olá, mundo!') # 파일 꼭대기에 # coding: cp1252 주석을 달아 SyntaxError를 해결할 수 있다

Olá, mundo!


이런 방식의 비아스키 식별자도 사용이 가능하지만 모든 사람이 알아볼 수 있도록 영문 식별자를 사용하는것이 좋다

In [10]:
ação = 'PBR' # ação = stock
Ɛ = 10**-6 # Ɛ = epsilon

### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법

chardet 패키지를 사용하면 인코딩 방식을 유추해낼 수 있다

In [11]:
!chardetect zen.txt

zen.txt: ascii with confidence 1.0


### 4.4.5 BOM: 유용한 깨진 문자

In [12]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

b'\xff\xfe' 문자가 앞에 나와있다. 이 문자가 바로 BOM(바이트 순서 표시)로 인코뎅한 인텔 CPU의 리틀엔디언 바이트 순서를 나타낸다

코드 포인트가 U+0045 (십진수 69)인 'E'문자는 다음과 같이 69와 0으로 인코딩되었다(list(u16)[2:4]) (빅엔디언 컴퓨터에서는 인코딩 순서가 반대가되어 0과 69로 인코딩된다)

In [13]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

UTF-16 인코딩은 특수문자를 텍스트 앞에 붙힌다. 해당 문자는 b'\xff\xfe'로 인코딩되며 UTF-16에 U+FFFE에 해당하는 문자가 없으므로 b'\xff\xfe' 문자는 출력되지 않는다 (ZERO WIDTH NO-BREAK SPACE)

UTF-16과 다르게 리틀엔디언을 명시하는 UTF-16LE와 빅엔디언을 명시하는 UTF-16BE로 변형하면, BOM을 생성하지 않는다

In [14]:
u16le = 'El Niño'.encode('utf_16le')
print(list(u16le))

u16be = 'El Niño'.encode('utf_16be')
print(list(u16be))

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]


## 4.5 텍스트 파일 다루기

In [15]:
# 예제 4-9 플랫폼 인코딩 문제 (지금 실습을 진행하는 pc가 우분투여서 그런지 버그가 발생하지 않음)
open('cafe.txt', 'w', encoding='utf_8').write('café')
open('cafe.txt').read()

'café'

In [16]:
# 예제 4-10 [예제 4-9]를 윈도우에서 실행해서 조사하면 버그와 해결 방법을 찾을 수 있다
fp = open('cafe.txt', 'w', encoding='utf_8')
print(fp) # open()함수는 텍스트 모드로 작동하며 TextIOWrapper 객체를 반환
print(fp.write('café')) # 지정한 문자의 유니코드 문자 수를 반환
fp.close()

import os
print(os.stat('cafe.txt').st_size) # 파일이 5 바이트라고 알려준다 (é가 2바이트)
fp2 = open('cafe.txt', encoding='cp1252')
print(fp2) # 내 환경에서는 UTF-8로 인코딩됨 => 실습을 위해 'cp1252'지정
print(fp2.encoding)
print(fp2.read()) # é가 제대로 해석되지 않음

fp3 = open('cafe.txt', encoding='utf_8') # 올바른 인코딩으로 연다
print(fp3)
print(fp3.read()) # 올바르게 해석

fp4 = open('cafe.txt', 'rb') # 'rb' 플래그는 파일으 이진 모드로 읽도록 한다
print(fp4)
print(fp4.read()) # 이 파일을 열면 bytes가 반환된다

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf_8'>
4
5
<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp1252'>
cp1252
cafÃ©
<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf_8'>
café
<_io.BufferedReader name='cafe.txt'>
b'caf\xc3\xa9'


### 4.5.1 기본 인코딩 설정: 정신 나간거 아냐?

내 실습환경은 ubuntu 이다 window의 경우에는 기본적으로 'cp1252'으로 인코딩 할 것이다

기존 인코딩에 의존하지 않도록 조심하자

In [17]:
# 예제 4-11 인코딩 기본값 알아보기
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
"""

my_file = open('cafe.txt', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value)) 

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


## 4.6 제대로 비교하기 위해 유니코드 정규화하기